**Problem statement:** 

Continuous bag of words (cbow) word2vec word embedding work is that it tends to predict the probability of a word given a context. A context may be a single word or a group of words. But for simplicity, I will take a single context word and try to predict a single target word. 

The purpose of this assignment is to be able to create a word embedding for the  given data set.  

**Data set :** w2v.txt 

In [1]:
import numpy as np

# Deep learning: 
from keras.models import Input, Model
from keras.layers import Dense

### Function Definations

In [2]:
import re
def create_unique_word_dict(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict    

def text_preprocessing(
    text:list,
    punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_“~''',
    stop_words=['and', 'a', 'is', 'the', 'in', 'be', 'will']
    )->list:
    
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")

    # Removing words that have numbers in them
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing digits
    text = re.sub(r'[0-9]+', '', text)

    # Cleaning the whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x!='']

    # Droping stop words
    text = [x for x in text if x not in stop_words]

    return text

# Functions to find the most similar word 
def euclidean(vec1:np.array, vec2:np.array) -> float:
    """
    A function to calculate the euclidean distance between two vectors
    """
    return np.sqrt(np.sum((vec1 - vec2)**2))

def find_similar(word:str, embedding_dict:dict, top_n=10)->list:
    """
    A method to find the most similar word based on the learnt embeddings
    """
    dist_dict = {}
    word_vector = embedding_dict.get(word, [])
    if len(word_vector) > 0:
        for key, value in embedding_dict.items():
            if key!=word:
                dist = euclidean(word_vector, value)
                dist_dict.update({
                    key: dist
                })

        return sorted(dist_dict.items(), key=lambda x: x[1])[0:top_n] 

In [3]:
# Reading the text from the input folder

file = open('w2v.txt', 'r')
texts = [line.strip() for line in file.readlines()]

In [4]:
# Defining the window for context
window = 2

# Creating a placeholder for the scanning of the word list. Cretion of CBOW
word_lists = []
all_text = []

for text in texts:

    # Cleaning the text
    text = text_preprocessing(text)

    # Appending to the all text list (Appending, "+" operator appends here as the all_text is a list)
    all_text += text 

    # Creating a context dictionary (Continous Bag Of Words)
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words by checking the boundry conditions.
            if i + 1 + w < len(text): 
                word_lists.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words by checking the boundry conditions.    
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])

In [5]:
unique_word_dict = create_unique_word_dict(all_text)

# Defining the number of features (unique words)
n_words = len(unique_word_dict)

# Getting all the unique words 
words = list(unique_word_dict.keys())

##### One Hot Encoding

In [6]:
# Creating the X and Y matrices using one hot encoding
X = []
Y = []

for i, word_list in enumerate(word_lists):
    # Getting the indices
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    # Creating the placeholders   
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words 
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

In [7]:
# Converting the X and Y to numpy arrays
X = np.array(X)
Y = np.array(Y)

# Defining the size of the embedding
embed_size = 8

##### Let's check the dimentions

In [8]:
#Total Combinations (pairs as window size was 2) created are:
np.array(word_lists).shape

(1694, 2)

In [9]:
(X.shape[0], Y.shape[0])

(1694, 1694)

In [10]:
(X.shape[1], Y.shape[1])

(255, 255)

In [11]:
# Total Number of words (Features)
n_words

255

##### Here the total number of features are 255 and records are 1694

In [12]:
# Defining the neural network

model_input = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(model_input) # Single (shallow) layer
x = Dense(units=Y.shape[1], activation='softmax')(x) # Output layer
model = Model(inputs=model_input, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

2022-04-28 21:03:33.272436: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Fit the model

number_of_epochs = 2000
model.fit(x = X, y = Y, batch_size = 256, epochs = number_of_epochs)

2022-04-28 21:03:34.463449: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2000
7/7 [==============================] - 1s 3ms/step - loss: 5.5402
Epoch 2/2000
7/7 [==============================] - 0s 5ms/step - loss: 5.5353
Epoch 3/2000
7/7 [==============================] - 0s 4ms/step - loss: 5.5308
Epoch 4/2000
7/7 [==============================] - 0s 5ms/step - loss: 5.5263
Epoch 5/2000
7/7 [==============================] - 0s 5ms/step - loss: 5.5218
Epoch 6/2000
7/7 [==============================] - 0s 6ms/step - loss: 5.5170
Epoch 7/2000
7/7 [==============================] - 0s 11ms/step - loss: 5.5121
Epoch 8/2000
7/7 [==============================] - 0s 4ms/step - loss: 5.5069
Epoch 9/2000
7/7 [==============================] - 0s 4ms/step - loss: 5.5014
Epoch 10/2000
7/7 [==============================] - 0s 3ms/step - loss: 5.4955
Epoch 11/2000
7/7 [==============================] - 0s 5ms/step - loss: 5.4892
Epoch 12/2000
7/7 [==============================] - 0s 4ms/step - loss: 5.4824
Epoch 13/2000
7/7 [=============================

7/7 [==============================] - 0s 5ms/step - loss: 4.6541
Epoch 104/2000
7/7 [==============================] - 0s 6ms/step - loss: 4.6452
Epoch 105/2000
7/7 [==============================] - 0s 5ms/step - loss: 4.6362
Epoch 106/2000
7/7 [==============================] - 0s 5ms/step - loss: 4.6272
Epoch 107/2000
7/7 [==============================] - 0s 5ms/step - loss: 4.6182
Epoch 108/2000
7/7 [==============================] - 0s 4ms/step - loss: 4.6090
Epoch 109/2000
7/7 [==============================] - 0s 4ms/step - loss: 4.5997
Epoch 110/2000
7/7 [==============================] - 0s 5ms/step - loss: 4.5906
Epoch 111/2000
7/7 [==============================] - 0s 8ms/step - loss: 4.5813
Epoch 112/2000
7/7 [==============================] - 0s 20ms/step - loss: 4.5719
Epoch 113/2000
7/7 [==============================] - 0s 8ms/step - loss: 4.5626
Epoch 114/2000
7/7 [==============================] - 0s 5ms/step - loss: 4.5531
Epoch 115/2000
7/7 [======================

7/7 [==============================] - 0s 4ms/step - loss: 3.7730
Epoch 205/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.7665
Epoch 206/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.7600
Epoch 207/2000
7/7 [==============================] - 0s 3ms/step - loss: 3.7536
Epoch 208/2000
7/7 [==============================] - 0s 3ms/step - loss: 3.7473
Epoch 209/2000
7/7 [==============================] - 0s 3ms/step - loss: 3.7409
Epoch 210/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.7347
Epoch 211/2000
7/7 [==============================] - 0s 3ms/step - loss: 3.7286
Epoch 212/2000
7/7 [==============================] - 0s 3ms/step - loss: 3.7224
Epoch 213/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.7162
Epoch 214/2000
7/7 [==============================] - 0s 3ms/step - loss: 3.7102
Epoch 215/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.7042
Epoch 216/2000
7/7 [=======================

7/7 [==============================] - 0s 4ms/step - loss: 3.3241
Epoch 306/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.3211
Epoch 307/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.3183
Epoch 308/2000
7/7 [==============================] - 0s 5ms/step - loss: 3.3155
Epoch 309/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.3126
Epoch 310/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.3098
Epoch 311/2000
7/7 [==============================] - 0s 3ms/step - loss: 3.3070
Epoch 312/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.3042
Epoch 313/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.3014
Epoch 314/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.2987
Epoch 315/2000
7/7 [==============================] - 0s 4ms/step - loss: 3.2960
Epoch 316/2000
7/7 [==============================] - 0s 3ms/step - loss: 3.2934
Epoch 317/2000
7/7 [=======================

...

7/7 [==============================] - 0s 5ms/step - loss: 2.5808
Epoch 1513/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5807
Epoch 1514/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5805
Epoch 1515/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5802
Epoch 1516/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5801
Epoch 1517/2000
7/7 [==============================] - 0s 16ms/step - loss: 2.5799
Epoch 1518/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5798
Epoch 1519/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5796
Epoch 1520/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5794
Epoch 1521/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5793
Epoch 1522/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5791
Epoch 1523/2000
7/7 [==============================] - 0s 3ms/step - loss: 2.5788
Epoch 1524/2000
7/7 [==========

Epoch 1612/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5642
Epoch 1613/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5641
Epoch 1614/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5638
Epoch 1615/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5638
Epoch 1616/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5636
Epoch 1617/2000
7/7 [==============================] - 0s 11ms/step - loss: 2.5633
Epoch 1618/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5632
Epoch 1619/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5631
Epoch 1620/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5630
Epoch 1621/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5628
Epoch 1622/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5627
Epoch 1623/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5626
Epoch 1624/2000

7/7 [==============================] - 0s 4ms/step - loss: 2.5496
Epoch 1712/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5493
Epoch 1713/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5492
Epoch 1714/2000
7/7 [==============================] - 0s 7ms/step - loss: 2.5490
Epoch 1715/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5488
Epoch 1716/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5487
Epoch 1717/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5486
Epoch 1718/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5484
Epoch 1719/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5482
Epoch 1720/2000
7/7 [==============================] - 0s 8ms/step - loss: 2.5481
Epoch 1721/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5480
Epoch 1722/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5479
Epoch 1723/2000
7/7 [===========

7/7 [==============================] - 0s 5ms/step - loss: 2.5358
Epoch 1812/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5357
Epoch 1813/2000
7/7 [==============================] - 0s 7ms/step - loss: 2.5354
Epoch 1814/2000
7/7 [==============================] - 0s 6ms/step - loss: 2.5354
Epoch 1815/2000
7/7 [==============================] - 0s 7ms/step - loss: 2.5352
Epoch 1816/2000
7/7 [==============================] - 0s 6ms/step - loss: 2.5354
Epoch 1817/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5350
Epoch 1818/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5348
Epoch 1819/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5348
Epoch 1820/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5346
Epoch 1821/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5345
Epoch 1822/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5344
Epoch 1823/2000
7/7 [===========

7/7 [==============================] - 0s 4ms/step - loss: 2.5235
Epoch 1912/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5232
Epoch 1913/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5231
Epoch 1914/2000
7/7 [==============================] - 0s 6ms/step - loss: 2.5230
Epoch 1915/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5230
Epoch 1916/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5228
Epoch 1917/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5228
Epoch 1918/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5226
Epoch 1919/2000
7/7 [==============================] - 0s 5ms/step - loss: 2.5225
Epoch 1920/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5223
Epoch 1921/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5222
Epoch 1922/2000
7/7 [==============================] - 0s 4ms/step - loss: 2.5221
Epoch 1923/2000
7/7 [===========

In [14]:
# Obtaining the weights from the neural network. 
# These are the word embeddings

# The input layer 
weights = model.get_weights()[0]

In [15]:
weights

array([[-2.4876907 , -1.7290698 , -0.05908765, ...,  0.45537218,
        -1.6656847 ,  2.298792  ],
       [-0.77696556, -1.781083  ,  0.8599954 , ..., -1.4655944 ,
        -1.5391856 ,  3.3372183 ],
       [-1.3029035 ,  2.2375948 ,  3.309436  , ...,  0.56699324,
         0.00495277, -1.7593904 ],
       ...,
       [ 0.30918783,  0.7676321 , -0.29727894, ..., -0.21534762,
        -0.05428566,  0.33367896],
       [-0.14472027, -0.42547026, -1.0240535 , ...,  1.1555617 ,
        -2.3558698 , -0.6266582 ],
       [ 1.7062875 , -0.59349245, -0.9998265 , ..., -1.8047333 ,
         3.4753835 , -1.1145939 ]], dtype=float32)

In [16]:
# Creating a dictionary to store the embeddings in. The key is a unique word and 
# the value is the numeric vector
embedding_dict = {}
for word in words: 
    embedding_dict.update({word: weights[unique_word_dict.get(word)]})

In [17]:
words_to_find_similar = ['stochastic', 'advances', 'allennlps', 'aminoacid', 'application', 'biochemical', 
                         'bioinformatics', 'calculate', 'characterize', 'clusters', 'data']

In [18]:
for wrd in words_to_find_similar:
    print(wrd, ' : ', find_similar(word=wrd, embedding_dict=embedding_dict, top_n=1)[0])

stochastic  :  ('neighbour', 3.0561986)
advances  :  ('partly', 3.3256738)
allennlps  :  ('elmofasttext', 2.9507635)
aminoacid  :  ('genevec', 2.0081208)
application  :  ('examples', 3.4962943)
biochemical  :  ('biophysical', 3.3159792)
bioinformatics  :  ('for', 3.3373694)
calculate  :  ('also', 3.149297)
characterize  :  ('biological', 2.9476185)
clusters  :  ('linguistics', 0.14611511)
data  :  ('underlying', 2.549056)
